<a href="https://colab.research.google.com/github/nupursjsu/Deep-Learning/blob/master/Ungraded_Assignment5/Keras_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Text Classification on IMDB dataset

## Importing necessary libraries

In [1]:
import numpy as np
import tensorflow as tf
!pip install tensorflow-hub
!pip install tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

     |████████████████████████████████| 3.3MB 53kB/s 
  Found existing installation: tensorflow-metadata 0.21.2
    Uninstalling tensorflow-metadata-0.21.2:
      Successfully uninstalled tensorflow-metadata-0.21.2
Version:  2.2.0
Eager mode:  True
Hub version:  0.8.0
GPU is available


## Loading and Exploring IMDB dataset

In [0]:
#Splitting the data into train, test and validation dataset
train_data, val_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

In [4]:
#Displaying the first 10 examples
train_examples, train_labels = next(iter(train_data.batch(10)))
train_examples

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [5]:
#Displaying first 10 labels
train_labels

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

## Building the model


In [6]:
#Creating a Keras layer that uses a TensorFlow Hub model to embed the sentences
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [7]:
#Buiding the full model
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


## Compiling the model

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Training the model


In [9]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=val_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 2s 75ms/step - loss: 1.0170 - accuracy: 0.4996 - val_loss: 0.8047 - val_accuracy: 0.5341
Epoch 2/20
30/30 [==============================] - 2s 73ms/step - loss: 0.7366 - accuracy: 0.5687 - val_loss: 0.6833 - val_accuracy: 0.6046
Epoch 3/20
30/30 [==============================] - 2s 75ms/step - loss: 0.6448 - accuracy: 0.6271 - val_loss: 0.6219 - val_accuracy: 0.6437
Epoch 4/20
30/30 [==============================] - 2s 76ms/step - loss: 0.5947 - accuracy: 0.6644 - val_loss: 0.5826 - val_accuracy: 0.6745
Epoch 5/20
30/30 [==============================] - 2s 73ms/step - loss: 0.5581 - accuracy: 0.6926 - val_loss: 0.5515 - val_accuracy: 0.7011
Epoch 6/20
30/30 [==============================] - 2s 73ms/step - loss: 0.5243 - accuracy: 0.7207 - val_loss: 0.5225 - val_accuracy: 0.7328
Epoch 7/20
30/30 [==============================] - 2s 73ms/step - loss: 0.4914 - accuracy: 0.7545 - val_loss: 0.4935 - val_accuracy: 0.7600
Epoch 8/20
30

## Evaluating the model

In [10]:
res = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, res):
  print("%s: %.3f" % (name, value))

49/49 - 2s - loss: 0.3212 - accuracy: 0.8549
loss: 0.321
accuracy: 0.855


It can se been that the model achieved an accuracy of aprox `85%` which is fairly good.